In [118]:
import pandas as pd
import requests
import json
import time
from dotenv import load_dotenv
import os
from bs4 import BeautifulSoup
import mimetypes

import re
from openai import OpenAI

In [119]:
load_dotenv('../tokens.env')

True

In [120]:
directory = pd.read_csv('../data/HD2024.csv')

In [121]:
def search_directory(query):
    query = query.lower()
    term_checker = lambda row: query in row.lower() if type(row) == str else False

    search_results = pd.concat([
        directory[directory['INSTNM'].apply(term_checker)],
        directory[directory['IALIAS'].apply(term_checker)]
    ]).drop_duplicates().reset_index(drop=True)

    return search_results

In [122]:
def search_directory_ui(query):
    temp = search_directory(query)
    return temp[['UNITID', 'INSTNM', 'STABBR']].to_dict(orient='records')

In [123]:
def param_string(parameters):
    string = ''
    for key in parameters:
        string += '&' + str(key) + '=' + str(parameters[key])
    return string

def assemble_query(parameters):
    url = f'https://customsearch.googleapis.com/customsearch/v1?cx={os.getenv("ENGINE_ID")}&key={os.getenv("API_KEY")}'
    url += param_string(parameters)
    url = url.replace(' ','+')
    return url

def send_query(query):
    data = requests.get(query)
    time.sleep(1)
    if data.status_code == 200:
        data = data.json()
        if 'items' in data:
            results = pd.DataFrame(data['items'])
            results['Query'] = query
            return results
    else:
        raise ValueError(f'query failed with status: {data.status_code}')

In [134]:
def retrieve_cds(unitid):
    curr_college = directory[directory['UNITID'].apply(lambda x: x == unitid)].reset_index(drop=True)
    domain = curr_college['WEBADDR'][0]

    # identify cds homepage
    query = assemble_query({
        'siteSearch': domain,
        'siteSearchFilter': 'i',
        'q': 'common data set'
    })

    results = send_query(query)
    cds_homepage = results['link'][0]

    # identify possible cds documents

    query = assemble_query({
        'siteSearch': domain,
        'siteSearchFilter': 'i',
        'q': 'common data set',
        'fileType': 'pdf'
    })

    results = send_query(query)

    results = results[results['title'].apply(lambda x: 'common' in x.lower())]

    return {
        'homepage': cds_homepage,
        'documents': results[['htmlTitle', 'link']].to_json(orient='records')
    }

In [125]:
def frame_url(url):
    if url is None:
        return ''
    else:
        return f'<a href="{url}">View Document</a>'

In [126]:
def retrieve_propublica_summary(unitid):
    curr_college = directory[directory['UNITID'].apply(lambda x: x == unitid)].reset_index(drop=True)
    ein = curr_college['EIN'][0]

    url = f'https://projects.propublica.org/nonprofits/api/v2/organizations/{ein}.json'

    data = requests.get(url).json()

    df = pd.DataFrame(data['filings_with_data'])

    df = df[['tax_prd_yr', 'totrevenue', 'totfuncexpns', 'totassetsend', 'totliabend', 'pdf_url']].rename({
        'tax_prd_yr': 'Year',
        'totrevenue': 'Total revenue',
        'totfuncexpns': 'Total expenses',
        'totassetsend': 'Total assets, end of year',
        'totliabend': 'Total liabilities, end of year',
        'pdf_url': 'Original Filing'
    }, axis=1)

    df['Original Filing'] = df['Original Filing'].apply(frame_url)

    return df.to_html()

In [127]:
def anchor_year_extract(anchor):
    match_obj = re.search('[0-9]{4}\-[0-9]{2,4}', anchor)
    if match_obj:
        return match_obj.group(0)
    else:
        return None

In [128]:
def anchor_url_extract(anchor):
    match_obj = re.search('href="([^"]+)', anchors[0])
    if match_obj:
        return match_obj.group(1)
    else:
        return None

experiments start here

ai experimentation

In [98]:
client = OpenAI()

In [99]:
prompt = f"""
Traverse this page, and return the raw html of all anchor tags containing urls that lead to a PDF file. Return the resulting tags as structured JSON under a key "urls". Return all tags that lead to PDF files, no exceptions. Return structured data only, no yapping. Page to traverse begins here: {hub_content}
"""

In [100]:
response = client.responses.create(
    model="gpt-4.1-nano",
    input=prompt
)

In [101]:
candidates = json.loads(response.output[0].content[0].text.replace('```', '').replace('json', ''))
anchors = pd.Series(candidates['urls'])

In [104]:
f'{cds_hub}/media/34940/download?inline'

'https://www.oberlin.edu/institutional-effectiveness/institutional-research-analytics/common-data-set/media/34940/download?inline'

In [103]:
anchors.apply(anchor_url_extract)[0]

'/media/34940/download?inline'

In [102]:
pd.DataFrame({
    'anchor': anchors,
    'year': anchors.apply(anchor_year_extract),
    'url': anchors.apply(anchor_url_extract)
})

,anchor,year,url
0,"<a class=""view-more"" href=""/media/34940/downlo...",2024-25,/media/34940/download?inline
1,"<a class=""view-more"" href=""/media/20096/downlo...",None,/media/34940/download?inline
2,"<a class=""view-more"" href=""/media/20097/downlo...",None,/media/34940/download?inline
3,"<a class=""view-more"" href=""/media/20013/downlo...",None,/media/34940/download?inline
4,"<a class=""view-more"" href=""/media/20014/downlo...",None,/media/34940/download?inline
5,"<a class=""view-more"" href=""/media/20015/downlo...",None,/media/34940/download?inline
6,"<a class=""view-more"" href=""/media/20016/downlo...",None,/media/34940/download?inline
